In [0]:
!pip install pyspark confluent-kafka

In [0]:
import json
import time
from datetime import datetime, timedelta
import random
from confluent_kafka import Producer

# Kafka configuration
conf = {
    'bootstrap.servers': 'pkc-619z3.us-east1.gcp.confluent.cloud:9092',  # Kafka cluster's bootstrap server
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'O3LJJAT5NL2RRB6G',  
    'sasl.password': 'wTc8QOLiu7ZU5PEXIUNB2BlHp23S5LLEJnOBbK4TeRWGjYVsDFHeN9RPYD8WN/Oo', 
}

In [0]:
import json
from datetime import datetime
from confluent_kafka import Consumer, KafkaException, KafkaError
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, TimestampType

# Kafka configuration
consumer = Consumer({
    'bootstrap.servers': 'pkc-619z3.us-east1.gcp.confluent.cloud:9092',  # Replace with your Kafka server
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism': 'PLAIN',
    'sasl.username': 'O3LJJAT5NL2RRB6G',  # Replace with your API Key
    'sasl.password': 'wTc8QOLiu7ZU5PEXIUNB2BlHp23S5LLEJnOBbK4TeRWGjYVsDFHeN9RPYD8WN/Oo',  # Replace with your API Secret
    'group.id': 'python-group-1',  # Consumer group ID
    'auto.offset.reset': 'earliest'  # Start reading messages from the beginning if no offset is committed
})

# Kafka topic
topic_name = 'stock'  # Using the provided topic name

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Real-Time Stock Price Analysis") \
    .getOrCreate()

# Define schema for stock data
schema = StructType([
    StructField("ticker", StringType(), True),
    StructField("price", FloatType(), True),
    StructField("timestamp", TimestampType(), True)
])

# Delta table path
delta_table_path = "/mnt/delta/stock_value"  # Make sure to use the correct path

# Subscribe to the topic with error handling
def subscribe_to_topic():
    try:
        # Attempt subscription to the topic
        consumer.subscribe([topic_name], on_assign=on_partition_assign)
        print(f"Successfully subscribed to topic: {topic_name}")
    except Exception as e:
        print(f"Error during subscription: {e}")

def on_partition_assign(consumer, partitions):
    """
    This function is called when partitions are assigned to the consumer.
    It's good for handling custom partition assignment logic.
    """
    print(f"Partitions assigned: {partitions}")
    for partition in partitions:
        print(f"Assigned partition {partition}")

def consume_messages():
    try:
        subscribe_to_topic()  # Subscribe to topic before starting consumption
        while True:
            msg = consumer.poll(1.0)  # Poll for new messages (1-second timeout)

            if msg is None:
                continue  # No message received
            if msg.error():
                if msg.error().code() == KafkaError._PARTITION_EOF:
                    # End of partition event
                    print(f"Reached end of partition: {msg.topic()} [{msg.partition()}]")
                elif msg.error():
                    raise KafkaException(msg.error())
            else:
                # Message successfully received
                message = json.loads(msg.value().decode('utf-8'))
                print(f"Consumed message: {message}")

                # Parse the timestamp string into a datetime object
                timestamp_str = message['timestamp']
                timestamp_obj = datetime.strptime(timestamp_str, '%Y-%m-%d %H:%M:%S')

                # Update message with the correct timestamp object
                message['timestamp'] = timestamp_obj

                # Create a DataFrame with the received message
                df = spark.createDataFrame([message], schema)

                # Show the dataframe (optional)
                df.show()

                # Append the data to Delta table
                df.write \
                  .format("delta").mode("append").save(delta_table_path)

                print(f"Appended data to Delta table: {delta_table_path}")

    except KeyboardInterrupt:
        print("Stopping consumer...")
    finally:
        # Close down consumer cleanly
        consumer.close()
        print("Consumer stopped.")

# Run the consumer
consume_messages()


Successfully subscribed to topic: stock
Partitions assigned: [TopicPartition{topic=stock,partition=0,offset=-1001,leader_epoch=None,error=None}, TopicPartition{topic=stock,partition=1,offset=-1001,leader_epoch=None,error=None}, TopicPartition{topic=stock,partition=2,offset=-1001,leader_epoch=None,error=None}, TopicPartition{topic=stock,partition=3,offset=-1001,leader_epoch=None,error=None}, TopicPartition{topic=stock,partition=4,offset=-1001,leader_epoch=None,error=None}, TopicPartition{topic=stock,partition=5,offset=-1001,leader_epoch=None,error=None}]
Assigned partition TopicPartition{topic=stock,partition=0,offset=-1001,leader_epoch=None,error=None}
Assigned partition TopicPartition{topic=stock,partition=1,offset=-1001,leader_epoch=None,error=None}
Assigned partition TopicPartition{topic=stock,partition=2,offset=-1001,leader_epoch=None,error=None}
Assigned partition TopicPartition{topic=stock,partition=3,offset=-1001,leader_epoch=None,error=None}
Assigned partition TopicPartition{to